In [3]:
import pandas as pd
import re
import torch

In [2]:
data_raw = pd.read_excel('./shopify reviews.xlsx')

In [4]:
data_raw.shape

(399, 10)

In [7]:
data_raw.head()

,app_id,author,rating,posted_at,body,helpful_count,helpful_count.1,developer_reply_posted_at,label,1 安全性（用户信息安全）
0,b1da53a4-0474-4700-9620-bf386bc033fb,Consuela,5,2020-08-06,NaN,0,NaN,NaT,6,2 易用性（UI设计美观合理便捷）
1,b1da53a4-0474-4700-9620-bf386bc033fb,L'Atelier Global,5,2020-08-04,"Still setting up my store, and after initially...",0,NaN,NaT,6,3 效率（服务器、多终端）
2,b1da53a4-0474-4700-9620-bf386bc033fb,city'super E-Shop,5,2020-08-04,"This is an excellent search app, which they ha...",0,NaN,NaT,7,4 兼容性（多地区、多语言兼容；与其他平台互通）
3,b1da53a4-0474-4700-9620-bf386bc033fb,PortableHandwashing.com,5,2020-07-30,"A+, great great great customer service! thanks...",0,NaN,NaT,7,5 应用程序基本功能（搜索功能、制作尺码表、聊天功能、即时品牌页面、程序运行快、删减产品、...
4,b1da53a4-0474-4700-9620-bf386bc033fb,ICCTUNING,5,2020-07-28,"I'm begginig to use this app, the search engin...",0,NaN,NaT,2,6 定制化功能


In [8]:
data2 = data_raw[['body', 'label']]

In [9]:
data2.shape

(399, 2)

In [10]:
data2.head()

,body,label
0,NaN,6
1,"Still setting up my store, and after initially...",6
2,"This is an excellent search app, which they ha...",7
3,"A+, great great great customer service! thanks...",7
4,"I'm begginig to use this app, the search engin...",2


In [12]:
data2 = data2.dropna()

In [13]:
data2.shape

(398, 2)

In [15]:
data2.head()

,body,label
1,"Still setting up my store, and after initially...",6
2,"This is an excellent search app, which they ha...",7
3,"A+, great great great customer service! thanks...",7
4,"I'm begginig to use this app, the search engin...",2
5,Great work guys. Easy app to install and use. ...,5


In [16]:
data = data2.copy()
data["body"] = data["body"].apply(lambda s : s.replace("\n", ' ').replace('\r', ' ').replace("\t", " "))
data.to_csv('train.tsv', sep = "\t", columns=['label', 'body'], index=0, header=0)

In [17]:
data.head()

,body,label
1,"Still setting up my store, and after initially...",6
2,"This is an excellent search app, which they ha...",7
3,"A+, great great great customer service! thanks...",7
4,"I'm begginig to use this app, the search engin...",2
5,Great work guys. Easy app to install and use. ...,5


In [18]:
test_data = data.loc[0:50]
test_data.to_csv('test.tsv', sep = "\t", columns=['label', 'body'], index=0, header=0)

## build dataset
### first step: define tokenize

In [21]:
# Split all the sentences into words
def my_tokenizer(s):
    s = s.replace(',', " ,").replace(".", " .").replace("?", " ?").replace("!", " !")
    return s.split()

my_tokenizer('Still setting up my store, and after initially paying for another search app, I made the decision to try Instant Search.   Their aesthetic really suits my type of store, and the custom settings pr...')

['Still',
 'setting',
 'up',
 'my',
 'store',
 ',',
 'and',
 'after',
 'initially',
 'paying',
 'for',
 'another',
 'search',
 'app',
 ',',
 'I',
 'made',
 'the',
 'decision',
 'to',
 'try',
 'Instant',
 'Search',
 '.',
 'Their',
 'aesthetic',
 'really',
 'suits',
 'my',
 'type',
 'of',
 'store',
 ',',
 'and',
 'the',
 'custom',
 'settings',
 'pr',
 '.',
 '.',
 '.']

### second step: define string clean

In [22]:
# No abbreviations
def clean_str(string):
    string = re.sub("[^A-Za-z0-9\-\?\!\.\,]", " ", string).lower()
    string = string.replace("that's", "that is")
    string = string.replace("isn't", "is not")
    string = string.replace("don't", "do not")
    string = string.replace("didn't", "did not")
    string = string.replace("won't", "will not")
    string = string.replace("can't", "can not")
    string = string.replace("you're", "you are")
    string = string.replace("they're", "they are")
    string = string.replace("you'll", "you will")
    string = string.replace("we'll", "we will")
    string = string.replace("what's", "what is")
    string = string.replace("i'm", "i am")
    string = string.replace("let's", "let us")
    return string

### third step: build vocab

In [26]:
from collections import Counter
from torchtext.vocab import vocab

def build_vocab(tokenizer, filepath, min_freq, specials=None):
    if specials is None:
        # UNK means unknown word, a word that doesn't exist the the vocabulary set
        # <PAD>, <EOS> and <GO> are all represented as <PAD> token. To complement the matrix given to be trained.
        specials = ['<unk>', '<pad>']
    counter = Counter()
    with open(filepath, encoding='utf8') as f:
        for string_ in f:
            string_ = string_.strip().split('\t')[-1][:-1] # strip removes spaces at the start and the end
            counter.update(tokenizer(clean_str(string_)))
    # torchtext.vocab.vocab(ordered_dict: Dict, min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) 
        # ordered_dict – Ordered Dictionary mapping tokens to their corresponding occurance frequencies.
        # min_freq – The minimum frequency needed to include a token in the vocabulary.
        # specials – Special symbols to add. The order of supplied tokens will be preserved.
        # special_first – Indicates whether to insert symbols at the beginning or at the end.
    return vocab(counter, min_freq=min_freq, specials=specials)

test_vocab = build_vocab(my_tokenizer, 'train.tsv', 1, ['<unk>', '<pad>'])
# index – Value of default index. This index will be returned when OOV token is queried. (OOV is out of vocabulary)
test_vocab.set_default_index(0)
test_vocab['gsdfg'] # so is 0 returned because of `test_vocab.set_default_index(0)`

0

In [27]:
def pad_sequence(sequences, batch_first=False, max_len=None, padding_value=0):
    max_size = sequences[0].size()
    trailing_dims = max_size[1:]
    length = max_len
    max_len = max([s.size(0) for s in sequences])
    if length is not None:
        max_len = max(length, max_len)
    if batch_first:
        out_dims = (len(sequences), max_len) + trailing_dims
    else:
        out_dims = (max_len, len(sequences)) + trailing_dims
    out_tensor = sequences[0].data.new(*out_dims).fill_(padding_value)
    for i, tensor in enumerate(sequences):
        length = tensor.size(0)
        # use index notation to prevent duplicate references to the tensor
        if batch_first:
            out_tensor[i, :length, ...] = tensor
        else:
            out_tensor[:length, i, ...] = tensor
    return out_tensor

### fourth step: trans Token sequence

In [29]:
s = ' and am so happy I found this app.'
s = clean_str(s)
tmp_vocab = build_vocab(my_tokenizer, filepath='./train.tsv', min_freq=1)
torch.tensor([tmp_vocab[token] for token in my_tokenizer(s)])

tensor([ 8, 38, 35, 42, 16, 43, 41, 15, 23])

In [30]:
from torch.utils.data import DataLoader


class LoadSentenceClassificationDataset():
    def __init__(self, train_file_path=None,  # 训练集路径
                 tokenizer=None,
                 batch_size=2,
                 min_freq=1,  # 最小词频，去掉小于min_freq的词
                 max_sen_len='same'):  # 最大句子长度，默认设置其长度为整个数据集中最长样本的长度
        # max_sen_len = None时，表示按每个batch中最长的样本长度进行padding
        # 根据训练预料建立字典
        self.tokenizer = tokenizer
        self.min_freq = min_freq
        self.specials = ['<unk>', '<pad>']
        self.vocab = build_vocab(self.tokenizer,filepath=train_file_path,
                                 min_freq=self.min_freq,specials=self.specials)
        self.vocab.set_default_index(0) # minimum is 1 so 0 is fine, otherwise use -1 as recommended
        self.PAD_IDX = self.vocab['<pad>']
        self.UNK_IDX = self.vocab['<unk>']
        self.batch_size = batch_size
        self.max_sen_len = max_sen_len

    def data_process(self, filepath):
        """
        将每一句话中的每一个词根据字典转换成索引的形式，同时返回所有样本中最长样本的长度
        :param filepath: 数据集路径
        :return:
        """
        raw_iter = iter(open(filepath, encoding="utf8"))
        data = []
        max_len = 0
        for raw in raw_iter:
            # BEFORE::: line = raw.rstrip("\n").split(",") ::: And it was csv!!!
            line = raw.rstrip("\n").split("\t")
            s, l = line[-1][:-1], line[0] # ⚠️ weird because this line truncated the string with comma, sometimes no label
            s = clean_str(s)
            tensor_ = torch.tensor([self.vocab[token] for token in
                                    self.tokenizer(s)], dtype=torch.long)
            l = torch.tensor(int(l) - 1, dtype=torch.long) # ⚠️ possible error
            max_len = max(max_len, tensor_.size(0)) # find the longest sentence
            data.append((tensor_, l)) # ⚠️ possible error
        return data, max_len

    def generate_batch(self, data_batch):
        batch_sentence, batch_label = [], []
        for (sen, label) in data_batch:  # 开始对一个batch中的每一个样本进行处理。
            batch_sentence.append(sen)
            batch_label.append(label)
        batch_sentence = pad_sequence(batch_sentence,  # [batch_size,max_len]
                                      padding_value=self.PAD_IDX,
                                      batch_first=False,
                                      max_len=self.max_sen_len)
        batch_label = torch.tensor(batch_label, dtype=torch.long)
        return batch_sentence, batch_label

    def load_train_val_test_data(self, train_file_paths, test_file_paths):
        train_data, max_sen_len = self.data_process(train_file_paths)  # 得到处理好的所有样本
        if self.max_sen_len == 'same':
            self.max_sen_len = max_sen_len
        test_data, _ = self.data_process(test_file_paths)
        train_iter = DataLoader(train_data, batch_size=self.batch_size,  # 构造DataLoader
                                shuffle=True, collate_fn=self.generate_batch) # complete the max length with padding
        test_iter = DataLoader(test_data, batch_size=self.batch_size,
                               shuffle=True, collate_fn=self.generate_batch)
        return train_iter, test_iter

dataset = LoadSentenceClassificationDataset('train.tsv', my_tokenizer)
tensor_data, max_len = dataset.data_process('train.tsv')
max_len

797

In [31]:
path = "train.tsv"
data_loader = LoadSentenceClassificationDataset(train_file_path=path,
                                                    tokenizer=my_tokenizer,
                                                    max_sen_len=None)
# max_sen_len = None时，表示按每个batch中最长的样本长度进行padding
# Feature is always 2, label and body
data, max_len = data_loader.data_process(path)
train_iter, test_iter = data_loader.load_train_val_test_data(path, path)
for sample, label in train_iter:
    print(sample.shape)  # [seq_len,batch_size]

torch.Size([33, 2])
torch.Size([246, 2])
torch.Size([49, 2])
torch.Size([28, 2])
torch.Size([118, 2])
torch.Size([67, 2])
torch.Size([58, 2])
torch.Size([25, 2])
torch.Size([81, 2])
torch.Size([30, 2])
torch.Size([49, 2])
torch.Size([62, 2])
torch.Size([55, 2])
torch.Size([180, 2])
torch.Size([120, 2])
torch.Size([76, 2])
torch.Size([40, 2])
torch.Size([94, 2])
torch.Size([67, 2])
torch.Size([69, 2])
torch.Size([19, 2])
torch.Size([49, 2])
torch.Size([30, 2])
torch.Size([50, 2])
torch.Size([82, 2])
torch.Size([32, 2])
torch.Size([37, 2])
torch.Size([32, 2])
torch.Size([60, 2])
torch.Size([41, 2])
torch.Size([29, 2])
torch.Size([95, 2])
torch.Size([25, 2])
torch.Size([36, 2])
torch.Size([86, 2])
torch.Size([125, 2])
torch.Size([95, 2])
torch.Size([44, 2])
torch.Size([45, 2])
torch.Size([67, 2])
torch.Size([51, 2])
torch.Size([94, 2])
torch.Size([29, 2])
torch.Size([61, 2])
torch.Size([60, 2])
torch.Size([141, 2])
torch.Size([165, 2])
torch.Size([78, 2])
torch.Size([111, 2])
torch.Size([

## network structure

In [33]:
from torch.nn.init import xavier_uniform_
import torch.nn.functional as F
from torch.nn import Parameter
import torch.nn as nn
import copy
import torch

In [35]:
is_print_shape = False

In [36]:
def multi_head_attention_forward(query,  # [tgt_len,batch_size, embed_dim]
                                 key,  # [src_len, batch_size, embed_dim]
                                 value,  # [src_len, batch_size, embed_dim]
                                 num_heads,
                                 dropout_p,
                                 out_proj,  # [embed_dim = vdim * num_heads, embed_dim = vdim * num_heads]
                                 training=True,
                                 key_padding_mask=None,  # [batch_size,src_len/tgt_len]
                                 q_proj=None,  # [embed_dim,kdim * num_heads]
                                 k_proj=None,  # [embed_dim, kdim * num_heads]
                                 v_proj=None,  # [embed_dim, vdim * num_heads]
                                 attn_mask=None,  # [tgt_len,src_len] or [num_heads*batch_size,tgt_len, src_len]
                                 ):
    q = q_proj(query)
    #  [tgt_len,batch_size, embed_dim] x [embed_dim,kdim * num_heads] = [tgt_len,batch_size,kdim * num_heads]

    k = k_proj(key)
    # [src_len, batch_size, embed_dim] x [embed_dim, kdim * num_heads] = [src_len, batch_size, kdim * num_heads]

    v = v_proj(value)
    # [src_len, batch_size, embed_dim] x [embed_dim, vdim * num_heads] = [src_len, batch_size, vdim * num_heads]
    if is_print_shape:
        print("" + "=" * 80)
        print("进入多头注意力计算:")
        print(
            f"\t 多头num_heads = {num_heads}, d_model={query.size(-1)}, d_k = d_v = d_model/num_heads={query.size(-1) // num_heads}")
        print(f"\t query的shape([tgt_len, batch_size, embed_dim]):{query.shape}")
        print(f"\t  W_q 的shape([embed_dim,kdim * num_heads]):{q_proj.weight.shape}")
        print(f"\t   Q  的shape([tgt_len, batch_size,kdim * num_heads]):{q.shape}")
        print("\t" + "-" * 70)

        print(f"\t  key 的shape([src_len,batch_size, embed_dim]):{key.shape}")
        print(f"\t  W_k 的shape([embed_dim,kdim * num_heads]):{k_proj.weight.shape}")
        print(f"\t   K  的shape([src_len,batch_size,kdim * num_heads]):{k.shape}")
        print("\t" + "-" * 70)

        print(f"\t value的shape([src_len,batch_size, embed_dim]):{value.shape}")
        print(f"\t  W_v 的shape([embed_dim,vdim * num_heads]):{v_proj.weight.shape}")
        print(f"\t   V  的shape([src_len,batch_size,vdim * num_heads]):{v.shape}")
        print("\t" + "-" * 70)
        print("\t ***** 注意，这里的W_q, W_k, W_v是多个head同时进行计算的. 因此，Q,K,V分别也是包含了多个head的q,k,v堆叠起来的结果 *****")

    tgt_len, bsz, embed_dim = query.size()  # [tgt_len,batch_size, embed_dim]
    src_len = key.size(0)
    head_dim = embed_dim // num_heads  # num_heads * head_dim = embed_dim
    scaling = float(head_dim) ** -0.5
    q = q * scaling  # [query_len,batch_size,kdim * num_heads]

    if attn_mask is not None:  # [tgt_len,src_len] or [num_heads*batch_size,tgt_len, src_len]
        if attn_mask.dim() == 2:
            attn_mask = attn_mask.unsqueeze(0)  # [1, tgt_len,src_len]
            if list(attn_mask.size()) != [1, query.size(0), key.size(0)]:
                raise RuntimeError('The size of the 2D attn_mask is not correct.')
        elif attn_mask.dim() == 3:
            if list(attn_mask.size()) != [bsz * num_heads, query.size(0), key.size(0)]:
                raise RuntimeError('The size of the 3D attn_mask is not correct.')
        # 现在 atten_mask 的维度就变成了3D

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    # [batch_size * num_heads,tgt_len,kdim]
    # 因为前面是num_heads个头一起参与的计算，所以这里要进行一下变形，以便于后面计算。 且同时交换了0，1两个维度
    k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)  # [batch_size * num_heads,src_len,kdim]
    v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)  # [batch_size * num_heads,src_len,vdim]
    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    # [batch_size * num_heads,tgt_len,kdim] x [batch_size * num_heads, kdim, src_len]
    # =  [batch_size * num_heads, tgt_len, src_len]  这就num_heads个QK相乘后的注意力矩阵

    if attn_mask is not None:
        attn_output_weights += attn_mask  # [batch_size * num_heads, tgt_len, src_len]

    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        # 变成 [batch_size, num_heads, tgt_len, src_len]的形状
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),  # 扩展维度，从[batch_size,src_len]变成[batch_size,1,1,src_len]
            float('-inf'))  #
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len,
                                                       src_len)  # [batch_size * num_heads, tgt_len, src_len]

    attn_output_weights = F.softmax(attn_output_weights, dim=-1)  # [batch_size * num_heads, tgt_len, src_len]
    attn_output_weights = F.dropout(attn_output_weights, p=dropout_p, training=training)
    attn_output = torch.bmm(attn_output_weights, v)
    # Z = [batch_size * num_heads, tgt_len, src_len]  x  [batch_size * num_heads,src_len,vdim]
    # = # [batch_size * num_heads,tgt_len,vdim]
    # 这就num_heads个Attention(Q,K,V)结果

    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    # 先transpose成 [tgt_len, batch_size* num_heads ,kdim]
    # 再view成 [tgt_len,batch_size,num_heads*kdim]
    attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)

    Z = out_proj(attn_output)
    # 这里就是多个z  线性组合成Z  [tgt_len,batch_size,embed_dim]
    if is_print_shape:
        print(f"\t 多头注意力中,多头计算结束后的形状（堆叠）为([tgt_len,batch_size,num_heads*kdim]){attn_output.shape}")
        print(f"\t 多头计算结束后，再进行线性变换时的权重W_o的形状为([num_heads*vdim, num_heads*vdim  ]){out_proj.weight.shape}")
        print(f"\t 多头线性变化后的形状为([tgt_len,batch_size,embed_dim]) {Z.shape}")
    return Z, attn_output_weights.sum(dim=1) / num_heads  # average attention weights over heads


In [37]:
class MyMultiheadAttention(nn.Module):
    """
    多头注意力机制的计算公式为（就是论文第5页的公式）：
    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O
        \text{where} head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
    """

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True):
        super(MyMultiheadAttention, self).__init__()
        """
        :param embed_dim:   词嵌入的维度，也就是前面的d_model参数，论文中的默认值为512
        :param num_heads:   多头注意力机制中多头的数量，也就是前面的nhead参数， 论文默认值为 8 // FREDHU heads => think of CNN filters
        :param dropout:
        :param bias:        最后对多头的注意力（组合）输出进行线性变换时，是否使用偏置
        """
        self.embed_dim = embed_dim  # 前面的d_model参数
        self.head_dim = embed_dim // num_heads  # head_dim 指的就是d_k,d_v
        self.kdim = self.head_dim
        self.vdim = self.head_dim

        self.num_heads = num_heads  # 多头个数
        self.dropout = dropout # FREDHU in case overfit

        assert self.head_dim * num_heads == self.embed_dim, "embed_dim 除以 num_heads必须为整数"
        # 上面的限制条件就是论文中的  d_k = d_v = d_model/n_head 条件

        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=bias)  # embed_dim = kdim * num_heads
        # 这里第二个维度之所以是embed_dim，实际上这里是同时初始化了num_heads个W_q堆叠起来的, 也就是num_heads个头
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=bias)  # W_k,  embed_dim = kdim * num_heads
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=bias)  # W_v,  embed_dim = vdim * num_heads
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        # 最后将所有的Z组合起来的时候，也是一次性完成， embed_dim = vdim * num_heads
        self._reset_parameters()

    def _reset_parameters(self):
        """
        以特定方式来初始化参数
        :return:
        """
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None):
        """
        在论文中，编码时query, key, value 都是同一个输入， 解码时 输入的部分也都是同一个输入，
        解码和编码交互时 key,value指的是 memory, query指的是tgt
        :param query: # [tgt_len, batch_size, embed_dim], tgt_len 表示目标序列的长度
        :param key:  #  [src_len, batch_size, embed_dim], src_len 表示源序列的长度
        :param value: # [src_len, batch_size, embed_dim], src_len 表示源序列的长度
        :param attn_mask: # [tgt_len,src_len] or [num_heads*batch_size,tgt_len, src_len]
                一般只在解码时使用，为了并行一次喂入所有解码部分的输入，所以要用mask来进行掩盖当前时刻之后的位置信息
        :param key_padding_mask: [batch_size, src_len], src_len 表示源序列的长度
        :return:
        attn_output: [tgt_len, batch_size, embed_dim]
        attn_output_weights: # [batch_size, tgt_len, src_len]
        """
        return multi_head_attention_forward(query, key, value, self.num_heads,
                                            self.dropout,
                                            out_proj=self.out_proj,
                                            training=self.training,
                                            key_padding_mask=key_padding_mask,
                                            q_proj=self.q_proj,
                                            k_proj=self.k_proj,
                                            v_proj=self.v_proj,
                                            attn_mask=attn_mask)

In [38]:
class MyTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super(MyTransformerEncoderLayer, self).__init__()
        """
        :param d_model:         d_k = d_v = d_model/nhead = 64, 模型中向量的维度，论文默认值为 512
        :param nhead:           多头注意力机制中多头的数量，论文默认为值 8
        :param dim_feedforward: 全连接中向量的维度，论文默认值为 2048
        :param dropout:         丢弃率，论文中的默认值为 0.1
        """
        self.self_attn = MyMultiheadAttention(d_model, nhead, dropout=dropout)

        # Implementation of Feedforward model
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)

        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.activation = F.relu # torch funtions

        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        """
        :param src: 编码部分的输入，形状为 [src_len,batch_size, embed_dim]
        :param src_mask:  编码部分输入的padding情况，形状为 [batch_size, src_len]
        :return:
        """
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask, )[0]  # 计算多头注意力
        # src2: [src_len,batch_size,num_heads*kdim] num_heads*kdim = embed_dim
        src = src + self.dropout1(src2)  # 残差连接
        src = self.norm1(src)  # [src_len,batch_size,num_heads*kdim]

        src2 = self.activation(self.linear1(src))  # [src_len,batch_size,dim_feedforward]
        src2 = self.linear2(self.dropout(src2))  # [src_len,batch_size,num_heads*kdim]
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src  # [src_len, batch_size, num_heads * kdim] <==> [src_len,batch_size,embed_dim]

def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [39]:
class MyTransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers, norm=None):
        super(MyTransformerEncoder, self).__init__()
        """
        encoder_layer: 就是包含有多头注意力机制的一个编码层
        num_layers: 克隆得到多个encoder layers 论文中默认为6
        norm: 归一化层
        """
        self.layers = _get_clones(encoder_layer, num_layers)  # 克隆得到多个encoder layers 论文中默认为6
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src, mask=None, src_key_padding_mask=None):
        """
        :param src: 编码部分的输入，形状为 [src_len,batch_size, embed_dim]
        :param mask:  编码部分输入的padding情况，形状为 [batch_size, src_len]
        :return:# [src_len, batch_size, num_heads * kdim] <==> [src_len,batch_size,embed_dim]
        """
        output = src
        for mod in self.layers:
            output = mod(output, src_mask=mask,
                         src_key_padding_mask=src_key_padding_mask)  # 多个encoder layers层堆叠后的前向传播过程
        if self.norm is not None:
            output = self.norm(output) # FREDHU standarlize
        return output  # [src_len, batch_size, num_heads * kdim] <==> [src_len,batch_size,embed_dim]

In [40]:
import math


class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model)) // FERDHU each i ocurs nhead times
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        #>>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)  # [max_len, d_model]
        """
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model)) // FERDHU each i ocurs nhead times
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        """
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # [max_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # [d_model/2] ::: start=0, end, step=1
        pe[:, 0::2] = torch.sin(position * div_term)  # [max_len, d_model/2]
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)  # [max_len, 1, d_model]
        # https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module.register_buffer
        self.register_buffer('pe', pe)

    def forward(self, x):  # [x_len, batch_size, d_model]
        """
        :param x: [x_len, batch_size, emb_size]
        :return: [x_len, batch_size, emb_size]
        """
        x = x + self.pe[:x.size(0), :]  # [batch_size, max_len, d_model]
        return self.dropout(x)


In [41]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    """
        :param tokens: shape : [len, batch_size]
        :return: shape: [len, batch_size, emb_size]
        """

    def forward(self, tokens): # ？？？？？
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size) # .long() == self.to(torch.int64)

In [42]:
class ClassificationModel(nn.Module):
    def __init__(self, vocab_size=None,
                 d_model=512, nhead=8,
                 num_encoder_layers=6,
                 dim_feedforward=2048,
                 dim_classification=64,
                 num_classification=10,
                 dropout=0.1):
        super(ClassificationModel, self).__init__()
        self.pos_embedding = PositionalEncoding(d_model=d_model, dropout=dropout)
        self.src_token_embedding = TokenEmbedding(vocab_size, d_model)
        encoder_layer = MyTransformerEncoderLayer(d_model, nhead,
                                                  dim_feedforward,
                                                  dropout)
        encoder_norm = nn.LayerNorm(d_model)
        self.encoder = MyTransformerEncoder(encoder_layer,
                                            num_encoder_layers, encoder_norm)
        self.classifier = nn.Sequential(nn.Linear(d_model, dim_classification),
                                        nn.Dropout(dropout),
                                        nn.Linear(dim_classification, num_classification))

    def forward(self,
                src,  # [src_len, batch_size]
                src_mask=None,
                src_key_padding_mask=None,  # [batsh_size, src_len]
                concat_type='sum'  # 解码之后取所有位置相加，还是最后一个位置作为输出
                ):
        src_embed = self.src_token_embedding(src)  # [src_len, batch_size, embed_dim]
        src_embed = self.pos_embedding(src_embed)  # [src_len, batch_size, embed_dim]
        memory = self.encoder(src=src_embed,
                              mask=src_mask,
                              src_key_padding_mask=src_key_padding_mask)
        # [src_len,batch_size,embed_dim]
        if concat_type == 'sum':
            memory = torch.sum(memory, dim=0)
        elif concat_type == 'avg':
            memory = torch.sum(memory, dim=0) / memory.size(0)
        else:
            memory = memory[-1, ::]  # 取最后一个时刻
        # [src_len, batch_size, num_heads * kdim] <==> [src_len,batch_size,embed_dim]
        out = self.classifier(memory)  # 输出logits
        return out  # [batch_size, num_class]

In [43]:
src_len = 7
batch_size = 2
dmodel = 32
num_head = 4
src = torch.tensor([[4, 3, 2, 6, 0, 0, 0],
                        [5, 7, 8, 2, 4, 0, 0]]).transpose(0, 1)  # 转换成 [src_len, batch_size]
src_key_padding_mask = torch.tensor([[True, True, True, True, False, False, False],
                                         [True, True, True, True, True, False, False]])
model = ClassificationModel(vocab_size=10, d_model=dmodel, nhead=num_head)
logits = model(src, src_key_padding_mask=src_key_padding_mask)
print(logits.shape) # torch.Size([2, 10])

torch.Size([2, 10])


In [44]:
logits

tensor([[ 0.9460,  3.1425, -0.6631, -1.2480,  2.4230,  1.3177,  0.2487, -0.3032,
         -1.1590,  0.6803],
        [ 2.6779,  3.3154, -0.1594, -1.6539, -2.3415,  0.9372, -3.3019, -1.2542,
          3.2723,  0.2104]], grad_fn=<AddmmBackward0>)

In [46]:
import time


class CustomSchedule(nn.Module):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = torch.tensor(d_model, dtype=torch.float32)
        self.warmup_steps = warmup_steps
        self.step = 1.

    def __call__(self):
        arg1 = self.step ** -0.5
        arg2 = self.step * (self.warmup_steps ** -1.5)
        self.step += 1.
        return (self.d_model ** -0.5) * min(arg1, arg2)

def train_model(config):
    data_loader = LoadSentenceClassificationDataset(config.train_corpus_file_paths,
                                                    my_tokenizer,
                                                    batch_size=config.batch_size,
                                                    min_freq=config.min_freq,
                                                    max_sen_len=config.max_sen_len)
    train_iter, test_iter = data_loader.load_train_val_test_data(
        config.train_corpus_file_paths, config.test_corpus_file_paths)

    classification_model = ClassificationModel(vocab_size=len(data_loader.vocab),
                                               d_model=config.d_model,
                                               nhead=config.num_head,
                                               num_encoder_layers=config.num_encoder_layers,
                                               dim_feedforward=config.dim_feedforward,
                                               dim_classification=config.dim_classification,
                                               num_classification=config.num_class,
                                               dropout=config.dropout)

    for p in classification_model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    model_save_path = os.path.join(config.model_save_dir, 'model.pt')
    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        classification_model.load_state_dict(loaded_paras)
        print("## 成功载入已有模型，进行追加训练......")
    classification_model = classification_model.to(config.device)
    loss_fn = torch.nn.CrossEntropyLoss()
    learning_rate = CustomSchedule(config.d_model) # FREDHU dynamic lr
    optimizer = torch.optim.Adam(classification_model.parameters(),
                                 lr=0.,
                                 betas=(config.beta1, config.beta2),
                                 eps=config.epsilon)
    classification_model.train()
    max_test_acc = 0
    for epoch in range(config.epochs):
        losses = 0
        start_time = time.time()
        for idx, (sample, label) in enumerate(train_iter):
            sample = sample.to(config.device)  # [src_len, batch_size]
            label = label.to(config.device)
            padding_mask = (sample == data_loader.PAD_IDX).transpose(0, 1)
            logits = classification_model(sample,
                                          src_key_padding_mask=padding_mask)  # [batch_size,num_class]
            optimizer.zero_grad()
            loss = loss_fn(logits, label)
            loss.backward()
            lr = learning_rate()
            for p in optimizer.param_groups:
                p['lr'] = lr
            optimizer.step()
            losses += loss.item()

            acc = (logits.argmax(1) == label).float().mean()
            if idx % 10 == 0:
                print(f"Epoch: {epoch}, Batch[{idx}/{len(train_iter)}], "
                      f"Train loss :{loss.item():.3f}, Train acc: {acc:.3f}")
        end_time = time.time()
        train_loss = losses / len(train_iter)
        print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s")
        if (epoch + 1) % config.model_save_per_epoch == 0:
            acc = evaluate(test_iter, classification_model, config.device)
            print(f"Accuracy on test {acc:.3f}, max acc on test {max_test_acc:.3f}")
            if acc > max_test_acc:
                max_test_acc = acc
                torch.save(classification_model.state_dict(), model_save_path)

def evaluate(data_iter, model, device):
    model.eval()
    with torch.no_grad():
        acc_sum, n = 0.0, 0
        for x, y in data_iter:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            acc_sum += (logits.argmax(1) == y).float().sum().item()
            n += len(y)
        model.train()
        return acc_sum / n

In [47]:
import os


class Config():
    """
    基于Transformer架构的类Translation模型配置类
    """

    def __init__(self):
        #   数据集设置相关配置
        self.project_dir = os.path.dirname(os.path.abspath('__file__'))
        self.dataset_dir = os.path.join(self.project_dir, '.')
        self.train_corpus_file_paths = os.path.join(self.dataset_dir, 'train.tsv')
        # print(self.train_corpus_file_paths)
        self.test_corpus_file_paths = os.path.join(self.dataset_dir, 'test.tsv')
        self.min_freq = 1
        self.max_sen_len = None

        #  模型相关配置
        self.batch_size = 128
        self.d_model = 512
        self.num_head = 8
        self.num_encoder_layers = 6
        self.num_decoder_layers = 6
        self.dim_feedforward = 512
        self.dim_classification = 256
        self.num_class = 10
        self.dropout = 0.1
        self.concat_type = 'avg'
        self.beta1 = 0.9
        self.beta2 = 0.98
        self.epsilon = 10e-9
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.epochs = 10
        self.model_save_dir = os.path.join(self.project_dir, 'cache')
        self.model_save_per_epoch = 2
        if not os.path.exists(self.model_save_dir):
            os.makedirs(self.model_save_dir)


In [48]:
config = Config()
train_model(config)

KeyboardInterrupt: 